LLM Benchmark

In [ ]:
from huggingface_hub import list_models
import time

nlp_tasks = {
    "Text Classification": "zero-shot-classification",
    "Named Entity Recognition (NER) OR Part-of-Speech Tagging (POS)": "token-classification",
    "Sentiment Analysis": "text-classification",
    "Semantic Analysis": "sentence-similarity",
    "Language Modeling OR Chatbots OR Text Generation": "text-generation",
    "Machine Translation": "translation",
    "Text Summarization": "summarization",
    "Information Extraction": "question-answering",
    "Speech Recognition": "automatic-speech-recognition",
}

print("🤖 Top 10 Models per NLP Task by trending_score and downloads")
print("=" * 70)


def get_models(pipeline_tag: str, sort: str, limit: int = 10):
    """Fetch models for a pipeline tag sorted by a metric (trending_score|downloads).

    Updated: Uses direct keyword args (library=, pipeline_tag=) instead of ModelFilter.
    """
    try:
        infos = list_models(
            filter="transformers", pipeline_tag=pipeline_tag, sort=sort, limit=limit
        )
        ids = []
        for info in infos:
            mid = getattr(info, "modelId", None) or getattr(info, "id", None)
            if mid:
                ids.append(mid)
        return ids
    except Exception as e:
        print(
            f"   ❌ Error fetching models for '{pipeline_tag}' sorted by '{sort}': {e}"
        )
        return []


def merged_unique(top_trending: list, top_downloads: list, limit: int = 10):
    """Merge two lists, keeping order priority by trending first then downloads,
    removing duplicates within the merged result, capped at limit.
    """
    seen = set()
    out = []
    for lst in (top_trending, top_downloads):
        for mid in lst:
            if mid not in seen:
                seen.add(mid)
                out.append(mid)
                if len(out) >= limit:
                    return out
    return out


start = time.time()
for task_name, pipeline_tag in nlp_tasks.items():
    print(f"\n📋 {task_name}  (tag: {pipeline_tag})")
    print("-" * 70)
    
    top_downloads = get_models(pipeline_tag, sort="downloads", limit=10)
    print("\nTop 10 by downloads:")
    if top_downloads:
        for i, model in enumerate(top_downloads, 1):
            print(f"   {i}. {model}")
    else:
        print("   No models found for downloads")

    top_trending = get_models(pipeline_tag, sort="trending_score", limit=10)
    print("Top 10 by trending_score:")
    if top_trending:
        for i, model in enumerate(top_trending, 1):
            print(f"   {i}. {model}")
    else:
        print("   No models found for trending_score")

    merged = merged_unique(top_downloads, top_trending, limit=20)
    print("\nMerged (unique up to 20):")
    if merged:
        for i, model in enumerate(merged, 1):
            print(f"   {i}. {model}")
    else:
        print("   No models found for merged")

    time.sleep(0.1)  # be gentle to the API

print("\n" + "=" * 70)
print(f"✅ Completed in {time.time() - start:.1f}s")

🤖 Top 10 Models per NLP Task by trending_score and downloads

📋 Text Classification  (tag: zero-shot-classification)
----------------------------------------------------------------------

Top 10 by downloads:
   1. facebook/bart-large-mnli
   2. sileod/deberta-v3-base-tasksource-nli
   3. MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli
   4. joeddav/xlm-roberta-large-xnli
   5. joeddav/bart-large-mnli-yahoo-answers
   6. MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7
   7. vicgalle/xlm-roberta-large-xnli-anli
   8. valhalla/distilbart-mnli-12-1
   9. cross-encoder/nli-deberta-v3-large
   10. MoritzLaurer/mDeBERTa-v3-base-mnli-xnli
Top 10 by trending_score:
   1. facebook/bart-large-mnli
   2. MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli
   3. joeddav/xlm-roberta-large-xnli
   4. MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli
   5. claritylab/zero-shot-explicit-binary-bert
   6. MoritzLaurer/deberta-v3-large-zeroshot-v1.1-all-33
   7. tasksource/ModernBERT-base-nli
  

In [ ]:
import time
import json
import os
from transformers import pipeline

TASK_NAME = "Named Entity Recognition (NER)"
PIPELINE_TAG = "token-classification"

# 1. Collect merged model list (top trending + top downloads, unique)
print(f"Collecting models for: {TASK_NAME} ({PIPELINE_TAG})")
trend = get_models(PIPELINE_TAG, sort="trending_score", limit=10)
download = get_models(PIPELINE_TAG, sort="downloads", limit=10)
merged_models = merged_unique(trend, download, limit=20)

print("\nMerged model list (up to 20 unique):")
for i, m in enumerate(merged_models, 1):
    print(f" {i:2d}. {m}")

# 2. Quick validation on a sample text
sample_text = (
    "Apple CEO Tim Cook met Elon Musk in Paris to discuss AI initiatives at the Louvre Museum. "
    "Microsoft and OpenAI representatives joined later in the afternoon."
)

# Limit number of models to validate to keep runtime/resource usage reasonable.
max_models = 20  # You can raise this after first quick check.
print(f"\nValidating first {max_models} models on sample text:\n{sample_text}\n")

try:
    import torch  # optional acceleration

    device = 0 if torch.cuda.is_available() else -1
except Exception:
    device = -1

results_summary = []

for model_id in merged_models[:max_models]:
    print("=" * 90)
    print(f"Loading model: {model_id}")
    t0 = time.time()
    try:
        try:
            ner_pipe = pipeline(
                PIPELINE_TAG,
                model=model_id,
                aggregation_strategy="simple",  # prefer grouped entities
                device=device,
            )
        except TypeError:
            # Fallback if aggregation_strategy not supported at init
            ner_pipe = pipeline(PIPELINE_TAG, model=model_id, device=device)

        outputs = ner_pipe(sample_text)
        elapsed = time.time() - t0
        print(f"Inference succeeded in {elapsed:.2f}s. Entities:")

        # Normalize output format
        normalized = []
        for ent in outputs:
            word = ent.get("word") or ent.get("entity")
            group = ent.get("entity_group") or ent.get("entity")
            score = ent.get("score", None)
            start = ent.get("start")
            end = ent.get("end")
            normalized.append(
                {
                    "word": word,
                    "entity_group": group,
                    "score": float(score) if score is not None else None,
                    "start": start,
                    "end": end,
                }
            )
            display_score = f"{score:.3f}" if score is not None else "-"
            print(
                f"  - {word:<20} {group:<18} score={display_score} span=({start},{end})"
            )

        results_summary.append(
            {
                "model": model_id,
                "elapsed_seconds": elapsed,
                "entities": normalized,
            }
        )
    except Exception as e:
        elapsed = time.time() - t0
        print(f"Failed after {elapsed:.2f}s: {e.__class__.__name__}: {e}")
        results_summary.append(
            {
                "model": model_id,
                "error": f"{e.__class__.__name__}: {e}",
                "elapsed_seconds": elapsed,
            }
        )

# 3. Persist results to disk
os.makedirs("results", exist_ok=True)
out_path = os.path.join("results", "ner_validation_results.json")
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(results_summary, f, ensure_ascii=False, indent=2)

print("\nSaved validation summary ->", out_path)
print("Done.")


Merged model list (up to 20 unique):
  1. dslim/bert-base-NER
  2. OpenMed/OpenMed-NER-ChemicalDetect-EuroMed-212M
  3. Babelscape/wikineural-multilingual-ner
  4. CAMeL-Lab/bert-base-arabic-camelbert-ca-pos-egy
  5. Jean-Baptiste/camembert-ner
  6. KoichiYasuoka/roberta-base-english-upos
  7. KoichiYasuoka/roberta-large-english-upos
  8. cahya/bert-base-indonesian-NER
  9. jiaqianjing/chinese-address-ner
 10. dslim/bert-large-NER
 11. w11wo/indonesian-roberta-base-posp-tagger
 12. dbmdz/bert-large-cased-finetuned-conll03-english
 13. adibvafa/CodonTransformer
 14. oliverguhr/fullstop-punctuation-multilang-large
 15. tsmatz/xlm-roberta-ner-japanese
 16. kredor/punctuate-all
 17. cahya/NusaBert-ner-v1.3
 18. OpenMed/OpenMed-NER-PharmaDetect-SuperClinical-434M
 19. Isotonic/distilbert_finetuned_ai4privacy_v2

Validating first 20 models on sample text:
Apple CEO Tim Cook met Elon Musk in Paris to discuss AI initiatives at the Louvre Museum. Microsoft and OpenAI representatives joined lat

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Inference succeeded in 3.20s. Entities:
  - Apple                ORG                score=0.998 span=(0,5)
  - Tim Cook             PER                score=1.000 span=(10,18)
  - Elon                 ORG                score=0.807 span=(23,27)
  - Mu                   PER                score=0.520 span=(28,30)
  - ##sk                 ORG                score=0.812 span=(30,32)
  - Paris                LOC                score=1.000 span=(36,41)
  - AI                   MISC               score=0.998 span=(53,55)
  - Lou                  LOC                score=0.984 span=(75,78)
  - ##vre Museum         LOC                score=0.983 span=(78,88)
  - Microsoft            ORG                score=0.999 span=(90,99)
  - OpenAI               ORG                score=0.991 span=(104,110)
Loading model: OpenMed/OpenMed-NER-ChemicalDetect-EuroMed-212M
Failed after 0.10s: ValueError: The repository OpenMed/OpenMed-NER-ChemicalDetect-EuroMed-212M contains custom code which must be executed

Device set to use cpu


Inference succeeded in 0.89s. Entities:
  - Apple                ORG                score=0.998 span=(0,5)
  - Tim Cook             PER                score=1.000 span=(10,18)
  - Elon Musk            PER                score=0.998 span=(23,32)
  - Paris                LOC                score=1.000 span=(36,41)
  - Louvre Museum        LOC                score=0.998 span=(75,88)
  - Microsoft            ORG                score=0.997 span=(90,99)
  - OpenAI               ORG                score=0.942 span=(104,110)
Loading model: CAMeL-Lab/bert-base-arabic-camelbert-ca-pos-egy


...

Inference succeeded in 2.66s. Entities:
  - Apple                ORG                score=0.814 span=(0,5)
  -  Tim Cook            PER                score=0.992 span=(9,18)
  -  Elon Musk           PER                score=0.989 span=(22,32)
  -  Paris               GPE                score=0.995 span=(35,41)
  -  AI                  PRD                score=0.777 span=(52,55)
  -  the Louvre Museum   LOC                score=0.821 span=(70,88)
  -  Microsoft           ORG                score=0.944 span=(89,99)
  -  OpenAI representatives ORG                score=0.805 span=(103,126)
Loading model: OpenMed/OpenMed-NER-PharmaDetect-SuperClinical-434M


Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Inference succeeded in 1.90s. Entities:
Loading model: Isotonic/distilbert_finetuned_ai4privacy_v2


Device set to use cpu


Inference succeeded in 0.52s. Entities:
  - tim                  FIRSTNAME          score=0.942 span=(10,13)
  - cook                 LASTNAME           score=0.979 span=(14,18)
  - elon                 FIRSTNAME          score=0.972 span=(23,27)
  - musk                 LASTNAME           score=0.965 span=(28,32)
  - paris                STATE              score=0.741 span=(36,41)

Saved validation summary -> results\ner_validation_results.json
Done.
